In [1]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster(threads_per_worker=1)

In [2]:
from dask.distributed import Client, wait
import time

import dask
import dask_cudf
import dask.dataframe as dd

import pandas as pd

import cudf
import numpy as np

import pandas.testing

from dask_cuml import knn as cumlKNN


In [3]:
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:40348 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 50.39 GB


In [4]:
def create_df(f, m, n):
    X = np.random.rand(m, n)
    ret = cudf.DataFrame([(i,X[:,i].astype(np.float32)) for i in range(n)], 
                        index = cudf.dataframe.RangeIndex(f*m, f*m+m, 1))
    
    print(str(ret))
    return ret

def get_meta(df):
    ret =  df.iloc[:0]
    return ret

In [5]:
workers = client.has_what().keys()
workers

dict_keys(['tcp://127.0.0.1:32881', 'tcp://127.0.0.1:39122'])

In [8]:
# Per gpu/worker
train_m = 1000000 
train_n = 1000

In [9]:
search_m = 10000
search_k = 15

In [10]:
%%time

# Create dfs on each worker (gpu)
dfs = [client.submit(create_df, n, train_m, train_n, workers = [worker])
       for worker, n in list(zip(workers, list(range(len(workers)))))]

# Wait for completion
wait(dfs)

meta = client.submit(get_meta, dfs[0]).result()

CPU times: user 2.71 s, sys: 438 ms, total: 3.15 s
Wall time: 33.9 s


In [11]:
lr = cumlKNN.KNN()

In [13]:
%%time
X_df = dask_cudf.from_delayed(dfs, meta=meta)

CPU times: user 1.01 s, sys: 77.8 ms, total: 1.09 s
Wall time: 1.04 s


In [15]:
%%time
lr.fit(X_df)

CPU times: user 393 ms, sys: 21.5 ms, total: 414 ms
Wall time: 2.4 s


In [16]:
%%time
I, D = lr.kneighbors(X_df[0:10000], search_k)

SORT: [((0, 10000), <Future: status: pending, key: get_I-c55d9feece4189b7005c32c14a7e9b15>, <Future: status: pending, key: get_D-a37e73e399e72e4c0fce423378c95dda>)]
CPU times: user 1.78 s, sys: 118 ms, total: 1.9 s
Wall time: 6.22 s


In [17]:
print(str(I.compute()))

   0        1        2        3        4        5        6 ...       14
0  0   940605   888960  1593386    54608   811882   383814 ...  1094105
1  1   383203  1940109  1152970  1043562   725757  1453459 ...   677345
2  2  1436609  1350775   850140   347937   919204  1460320 ...  1084498
3  3  1509633   796809  1083947  1025346  1187030  1998520 ...   477976
4  4  1983833   934084  1186005   444926  1084622  1313104 ...  1334360
5  5  1020549  1994321  1135094  1394097   814302    77392 ...  1519412
6  6   329919   277397   480477  1719748  1450437   759490 ...  1732677
7  7  1937790   130727    97211  1538184   807452   501294 ...  1907852
8  8   811021  1145868  1347154  1547789  1175363  1842441 ...   724036
9  9  1657225  1821883   646006  1130071   857546   489549 ...   489872
[9991 more rows]
[7 more columns]


In [18]:
print(str(D.compute()))

               0          1          2          3          4          5          6 ...         14
0            0.0  138.37921  139.42023  139.66168  139.66785  139.69376  139.94037 ...   141.6109
1            0.0  143.39581  143.72086  144.03378  144.24677  144.48993  144.82483 ...   145.7225
2            0.0  136.36343  136.89151  138.41376  138.52783  138.71863  139.21506 ...  139.77316
3  0.00012207031  138.44229  139.59036  139.93246  140.09909   140.5545   140.7428 ...  141.45444
4  0.00061035156  137.37439  138.03433  138.18713  138.42624   138.4707  138.76312 ...  140.00613
5            0.0  138.99628   140.8153  141.18555  141.48456  141.52322  142.31622 ...  144.11554
6  0.00030517578  136.46716   139.4671  139.82892  140.79016   141.7179    142.289 ...   142.8277
7            0.0   138.0683  138.53616  138.81369  138.97708  139.15689  139.22995 ...  140.15619
8            0.0  136.80609   137.6225  141.09644  141.53564    141.638  141.71655 ...  142.36172
9   0.0004272461  13

In [ ]:
a.index[-1]